In [1]:
# in case there are any problems with importing because path is wrong
import sys
sys.path.append('C:/Users/nerpa/Dropbox (Personal)/Research/discrete_sr/code/SPIDER_discrete')

In [2]:
import numpy as np

from commons.weight import *
from utils import save, load
from library import *
from process_library_terms import *

In [42]:
import netCDF4 as nc

dataset = 'viscek/N=100000_rho=2.000_v=0.500_alpha=6.000_delta=0.150_t0=10000000_tf=10100000_dt=0.010_T=100_g=0.nc'
#dataset = 'viscek/N=100000_rho=2.000_v=0.500_alpha=6.000_delta=0.200_t0=10000000_tf=10100000_dt=0.010_T=100_g=0.nc'
#dataset = 'viscek/N=100000_rho=2.000_v=0.500_alpha=6.000_delta=0.300_t0=10000000_tf=10100000_dt=0.010_T=100_g=0.nc'

# (don't interpolate)

ds = nc.Dataset(dataset)
print(ds)

nt = len(ds.dimensions["rec"])
n_particles = len(ds.dimensions["Nv"])
print(nt, n_particles)
deltat = 1
# deltat = 0.01
v = 0.5
positions = np.array(ds.variables["pos"]).T.astype('float64')
positions = np.reshape(positions, (n_particles, 2, nt))
positions -= np.min(positions[:]) # recenter positions to [0, L]
directors = np.array(ds.variables["director"]).astype('float64')
vs = v*np.stack([np.cos(directors), np.sin(directors)], axis=1).T
print(directors.shape)
print(positions.shape)
print(vs.shape)
dims = [np.max(positions[:]), nt]
world_size = np.array([dims[0], dims[0], dims[1]])
print(world_size)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): rec(1000), Nv(100000), dof(200000), boxdim(4), unit(1)
    variables(dimensions): float64 time(rec, unit), float64 means0(rec, unit), float64 pos(rec, dof), int32 type(rec, Nv), float64 director(rec, Nv), float64 BoxMatrix(rec, boxdim)
    groups: 
1000 100000
(1000, 100000)
(100000, 2, 1000)
(100000, 2, 1000)
[ 223.607  223.607 1000.   ]


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
plt.rcParams['animation.ffmpeg_path'] = 'C:/Users/nerpa/Downloads/ffmpeg-6.0-essentials_build/bin/ffmpeg.exe'

vid_file = 'viscek/eta0.15_t0.01.mp4'
#vid_file = 'viscek/eta0.2_t0.01.mp4'
#vid_file = 'viscek/eta0.3_t0.01.mp4'


fig, ax = plt.subplots(figsize=(6, 6))
qv = ax.quiver(positions[:, 0, -1], positions[:, 1, -1], 1, 0, clim=[-np.pi, np.pi])

def animate(i):
    if i % 10 == 0:
        print(i)
    qv.set_offsets(positions[:, :, i])
    norms = np.sqrt(vs[:, 0, i] ** 2 + vs[:, 1, i] ** 2)
    qv.set_UVC(vs[:, 0, i] / norms, vs[:, 1, i] / norms, np.angle(vs[:, 0, i] + 1.0j * vs[:, 1, i]))
    return qv,

anim = FuncAnimation(fig, animate, np.arange(0, positions.shape[-1]), interval=1, blit=True)
FFwriter = animation.FFMpegWriter(fps=24, codec="libx264")
anim.save(vid_file, writer=FFwriter, dpi=100)

0
0
0
10
20
30
40
50
60
70


In [ ]:
%%prun # profiling

data_dict = {}
data_dict['v'] = vs
v_obs = Observable('v', 1)
observables = [v_obs]

# fix random seed
np.random.seed(1)

# initial setup of dataset
corr_L = 5
kernel_sigma = corr_L/2.5
cg_res = 1
srd = SRDataset(world_size, data_dict, positions, observables, 
                kernel_sigma=kernel_sigma, cg_res=cg_res, deltat=deltat, cutoff=6)

# initialize libraries, domains, and weights
#srd.make_libraries(max_complexity=4, max_observables=3)
srd.make_libraries(max_complexity=5, max_observables=3, max_rho=2)
#srd.make_libraries(max_complexity=6, max_observables=4)

dom_width = 80
dom_time = 50 #previously 20 (without interpolation)
srd.make_domains(ndomains=15, domain_size=[dom_width, dom_width, dom_time], pad=kernel_sigma*8) #ndomains = 30
srd.make_weights(m=8, qmax=1)
srd.set_LT_scale(L=corr_L, T=corr_L/v) # note that this line must go before make_library_matrices
srd.make_library_matrices(debug=False)

save('Q_XY_t0.01_eta0.15.npy', srd.dxs, srd.libs)
#save('Q_XY_t0.01_eta0.2.npy', srd.dxs, srd.libs)
#save('Q_XY_t0.01_eta0.3.npy', srd.dxs, srd.libs)

In [ ]:
_, libs = load('Q_XY_t0.01_eta0.15.npy', 2)
#_, libs = load('Q_XY_t0.01_eta0.2.npy', 2)
#_, libs = load('Q_XY_t0.01_eta0.3.npy', 2)
libs = libs.item()

In [ ]:
from commons.sparse_reg_bf import *
from identify_models import *
import copy

# for regression we now need to construct a Scaler, Initializer, ModelIterator, and Threshold
scaler0 = Scaler(sub_inds=None, char_sizes=libs[0].col_weights, row_norms=None)
init0 = Initializer(method='combinatorial', start_k=2)
#init0 = Initializer(method='power', start_k=10)
#res0 = Residual(residual_type='fixed_column', anchor_col=0)
res0 = Residual(residual_type='dominant_balance')

iter0 = ModelIterator(max_k=10, backward_forward=True, brute_force=True) # test also boolean toggles
thres0 = Threshold(threshold_type='jump', gamma=1.5, n_terms=None)
#thres0 = Threshold(threshold_type='information', ic=AIC)
#thres0 = Threshold(threshold_type='jump', gamma=1.5, n_terms=3)

opts = {'scaler': scaler0, 'initializer': init0, 'residual': res0,
        'model_iterator': iter0, 'threshold': thres0}
opts['verbose'] = False
opts1 = copy.deepcopy(opts) # need to be careful to deep copy for the stateful ModelIterator
opts['inhomog'] = False
opts['inhomog_col'] = None
#opts['verbose'] = False
sub_inds1 = list(range(len(libs[1].terms)))
#sub_inds1.remove(35) # dt rho[v_i * v_j * v_j]
#sub_inds1.remove(21) # rho * dt rho[v_i]
opts1['scaler'] = Scaler(sub_inds=sub_inds1, char_sizes=libs[1].col_weights)
opts1['residual'] = Residual(residual_type='fixed_column', anchor_col=13)
opts1['threshold'] = Threshold(threshold_type='jump', gamma=1.5, n_terms=5)

#opts1['verbose'] = True
opts1['inhomog'] = True
opts1['inhomog_col'] = 13 
    
opts['verbose']=False
opts1['verbose']=False

# note that interleave_identify doesn't work with inhomog or fixed-column residual
opts1['inhomog'] = False
opts1['inhomog_col'] = None
opts1['residual'] = copy.deepcopy(opts['residual'])
opts1['threshold'] = Threshold(threshold_type='jump', gamma=1.5, n_terms=None)

eqs, lambdas, derived_eqs, excluded_terms = interleave_identify([libs[0].Q, libs[1].Q], 
[opts, opts1], [libs[0].terms, libs[1].terms], threshold=1e-1, experimental=True)

In [ ]:
opts1['inhomog'] = True
opts1['inhomog_col'] = 13 # dt rho[v_i]

remove_terms = [21, 35] # rho * dt rho[v_i], dt rho[v_i * v_j * v_j]
for term in remove_terms:
    if term in opts1['scaler'].sub_inds:
        opts1['scaler'].sub_inds.remove(term) 
opts1['scaler'].reset_inds(opts1['scaler'].sub_inds)

opts1['threshold'] = Threshold(threshold_type='jump', gamma=1.5, n_terms=2)
opts1['verbose'] = False
Xi, lambd, best_term, lambda1 = sparse_reg_bf(libs[1].Q, **opts1)
zipped = [(libs[1].terms[i], c) for i, c in enumerate(Xi) if c != 0]
print(Equation([e[0] for e in zipped], [e[1] for e in zipped]), "; residual:", lambd)

In [ ]:
import seaborn as sns
%matplotlib inline
sns.set(rc={'figure.figsize':(14,9)})

# calculate the correlation matrix
R2 = (np.corrcoef(libs[0].Q, rowvar=False)**2)
# plot the heatmap
label_names = [str(term) for term in libs[0].terms]
sns.heatmap(R2, xticklabels=label_names, yticklabels=label_names, annot=True, fmt=".2f")

In [22]:
for i, term, size in zip(list(range(len(libs[0].terms))), libs[0].terms, libs[0].col_weights):
    print(i, term, size, term.complexity)
for i, term, size in zip(list(range(len(libs[1].terms))), libs[1].terms, libs[1].col_weights):
    print(i, term, size, term.complexity)

0 1 1 0
1 rho 2.000000025293657 1
2 dj^2 rho 0.048849751257286744 3
3 dj^2 dk^2 rho 0.00195399005029147 5
4 dt rho 0.12212437814321686 2
5 dt dj^2 rho 0.004884975125728675 4
6 dt^2 rho 0.012212437814321686 3
7 dt^2 dj^2 rho 0.0004884975125728675 5
8 dt^3 rho 0.0012212437814321687 4
9 dt^4 rho 0.00012212437814321687 5
10 rho * rho 4.000000101174629 2
11 rho * dj^2 rho 0.09769950375016234 4
12 rho * dt rho 0.24424875937540586 3
13 rho * dt dj^2 rho 0.009769950375016235 5
14 rho * dt^2 rho 0.024424875937540586 4
15 rho * dt^3 rho 0.0024424875937540586 5
16 dj rho[v_j] 0.03495691833410024 3
17 dj^2 dk rho[v_k] 0.0013982767333640094 5
18 dt dj rho[v_j] 0.0034956918334100237 4
19 dt^2 dj rho[v_j] 0.00034956918334100236 5
20 rho * dj rho[v_j] 0.06991383755238877 4
21 rho * dt dj rho[v_j] 0.006991383755238877 5
22 rho[v_j * v_j] 0.2500000031617072 3
23 dj^2 rho[v_k * v_k] 0.0010006078704318264 5
24 dj dk rho[v_j * v_k] 0.0010006078704318264 5
25 dt rho[v_j * v_j] 0.002501519676079566 4
26 dt^2

In [ ]:
continuity_terms = ['dt rho', 'dj rho[v_j]']
#continuity_terms = ['dt rho', 'dj rho[v_j]', 'dt^2 rho', 'dj^2 rho']

#continuity_terms = ['1', 'rho', 'rho * rho']
#continuity_terms = ['1']
#continuity_terms = ['rho * dt rho', 'rho * dj rho[v_j]']
#continuity_terms = ['dt rho * rho', 'rho * rho * dj v_j', 'dj rho * rho * v_j']
#continuity_terms = ['dt rho', 'dj rv_j']
col_numbers = [find_term(libs[0].terms, name) for name in continuity_terms]
Xi, lambd = regress(libs[0].Q, col_numbers, libs[0].col_weights, normalization=opts['anchor_norm'])
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[0].terms[i]}. (Char. size: {libs[0].col_weights[i]})")
print("Model residual:", lambd)

In [ ]:
### plot strong-form fields

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
plt.rcParams['animation.ffmpeg_path'] = 'C:/Users/nerpa/Downloads/ffmpeg-6.0-essentials_build/bin/ffmpeg.exe'

vid_file = 'viz_test.mp3'

zero_arr = [0] * srd.n_dimensions
constant_weight = Weight(m=zero_arr, q=zero_arr, k=zero_arr) # should evaluate to 1 everywhere

def make_residual_arr(equation, domain, kc):
    arrs = [c*srd.make_tw_arr(term, constant_weight, [domain], kc, by_parts=False) 
            for term, c in zip(equation.term_list, equation.coeffs)]
    return sum(arrs)[..., 0]

def make_all_but_first(equation, domain, kc):
    arrs = [c*srd.make_tw_arr(term, constant_weight, [domain], kc, by_parts=False) 
            for term, c in zip(equation.term_list, equation.coeffs)]
    return sum(arrs[1:])[..., 0]

domain = srd.domains[0]
kc = 0 # can select x, y components

print(eqn)

#term1 = srd.libs[1].terms[find_term(libs[1].terms, 'dt rho[v_i]')]
#tw_arr = srd.make_tw_arr(term1, constant_weight, [domain], kc, by_parts=False)[..., 0]
term1 = srd.libs[0].terms[find_term(libs[0].terms, 'rho')]
tw_arr = srd.make_tw_arr(term1, constant_weight, [domain], None, by_parts=False)[..., 0]
fig, (ax1, ax2) = plt.subplots(2, 1)
im1 = ax1.imshow(tw_arr[:, :, 0])
#colorbar
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes("right", size="5%", pad=0.05)
cb1 = plt.colorbar(im1, cax=cax1)

tw_arr2 = make_residual_arr(eqn, domain, kc) # or 
im2 = ax2.imshow(tw_arr2[:, :, 0])
#colorbar
divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes("right", size="5%", pad=0.05)
cb2 = plt.colorbar(im2, cax=cax2)

ax1.set_ylabel(str(term1))
ax2.set_ylabel("residual")

def animate(i):
    if i % 10 == 0:
        print(f"frame={i}")
    im1.set_array(tw_arr[:, :, i])
    im2.set_array(tw_arr2[:, :, i])
    vmax1 = np.max(tw_arr[:, :, i])
    vmin1 = np.min(tw_arr[:, :, i])
    vmax2 = np.max(tw_arr2[:, :, i])
    vmin2 = np.min(tw_arr2[:, :, i])
    #im1.set_clim(vmin1, vmax1)
    #im2.set_clim(vmin2, vmax2)

    return im1, im2

anim = FuncAnimation(fig, animate, np.arange(0, len(domain.times)), interval=100, blit=True)
FFwriter = animation.FFMpegWriter(fps=24, codec="libx264")
#anim.save(vid_file, writer=FFwriter, dpi=100)
HTML(anim.to_html5_video())

#plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(tw_arr[0, 0, :])
ax.plot(tw_arr[1, 0, :])
ax.plot(tw_arr[2, 0, :])

In [11]:
constraint_terms = ['rho', 'rho[v_j * v_j]']
col_numbers = [find_term(libs[0].terms, name) for name in constraint_terms]
Xi, lambd = regress(libs[0].Q, col_numbers, libs[0].col_weights)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[0].terms[i]}. (Char. size: {libs[0].col_weights[i]})")
print("Model residual:", lambd)

[Term 1] -0.25000000000000017 * rho. (Char. size: 2.000000025293657)
[Term 22] 1.0 * rho[v_j * v_j]. (Char. size: 0.2500000031617072)
Model residual: 8.283271158830063e-16


In [12]:
wave_terms = ['dt^2 rho', 'dj^2 rho']
col_numbers = [find_term(libs[0].terms, name) for name in wave_terms]
Xi, lambd = regress(libs[0].Q, col_numbers, libs[0].col_weights)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[0].terms[i]}. (Char. size: {libs[0].col_weights[i]})")
print("Model residual:", lambd)

[Term 2] -0.13484239722442778 * dj^2 rho. (Char. size: 0.048849751257286744)
[Term 6] 1.0 * dt^2 rho. (Char. size: 0.012212437814321686)
Model residual: 0.4630823815284282


In [13]:
burger_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]']
#burger_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]', 'di rho']#, 'dj^2 rho[v_i]']# 'dj^2 rho[v_i]']
#burger_terms = ['dt rho[v_i]', 'rho * dj rho[v_i * v_j]', 'di rho']
col_numbers = [find_term(libs[1].terms, name) for name in burger_terms]
Xi, lambd = regress(libs[1].Q, col_numbers, libs[1].col_weights)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[1].terms[i]}. (Char. size: {libs[1].col_weights[i]})")
print("Model residual:", lambd)

[Term 13] 0.9873453145900928 * dt rho[v_i]. (Char. size: 0.01747845916705012)
[Term 24] 1.0 * dj rho[v_i * v_j]. (Char. size: 0.005003039352159132)
Model residual: 0.14586121110096015


In [14]:
#euler_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]']
euler_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]', 'di rho']
#euler_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]', 'di rho', 'dj^2 rho[v_i]']
#euler_terms = ['dj rho[v_i * v_j]', 'di rho']
col_numbers = [find_term(libs[1].terms, name) for name in euler_terms]
Xi, lambd = regress(libs[1].Q, col_numbers, libs[1].col_weights)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[1].terms[i]}. (Char. size: {libs[1].col_weights[i]})")
print("Model residual:", lambd)

[Term 0] 0.002511222354484635 * di rho. (Char. size: 0.24424875628643372)
[Term 13] 0.9987648071887012 * dt rho[v_i]. (Char. size: 0.01747845916705012)
[Term 24] 1.0 * dj rho[v_i * v_j]. (Char. size: 0.005003039352159132)
Model residual: 0.14552068875769886


In [15]:
toner_tu_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]', 'rho[v_i] * dj rho[v_j]', 'rho[v_i]',
                'di rho', 'dj^2 rho[v_i]', 'di dj rho[v_j]']#, 'dj dk rho[v_i * v_j * v_k]', 
                #'rho * di rho', 'rho * rho * di rho']
# third term is a bit sketchy
col_numbers = [find_term(libs[1].terms, name) for name in toner_tu_terms]
Xi, lambd = regress(libs[1].Q, col_numbers, libs[1].col_weights)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[1].terms[i]}. (Char. size: {libs[1].col_weights[i]})")
print("Model residual:", lambd)

[Term 0] 0.0038654497762913294 * di rho. (Char. size: 0.24424875628643372)
[Term 10] 0.00023545175394586962 * rho[v_i]. (Char. size: 0.7071067901292057)
[Term 11] 0.2941205541285419 * di dj rho[v_j]. (Char. size: 0.0069913836668200465)
[Term 12] -0.1523119907402973 * dj^2 rho[v_i]. (Char. size: 0.0069913836668200465)
[Term 13] 0.9582763922533301 * dt rho[v_i]. (Char. size: 0.01747845916705012)
[Term 24] 1.0 * dj rho[v_i * v_j]. (Char. size: 0.005003039352159132)
[Term 27] -0.021253094159674035 * rho[v_i] * dj rho[v_j]. (Char. size: 0.0247182743160344)
Model residual: 0.13116340333329035


In [16]:
#inhomog_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]']
#inhomog_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]', 'rho * dt rho[v_i]', 'rho * dj rho[v_i * v_j]']
#inhomog_terms = ['dt rho[v_i]', 'dt^2 di rho', 'rho * di rho', 'rho * dt di rho', 'rho * dt^2 di rho',
#                 'rho * dt^2 rho[v_i]']
inhomog_terms = ['dj rho[v_i * v_j]', 'dt di rho', 'dt dj^2 rho[v_i]', 'dt rho[v_i]', 'dt^2 di rho',
'rho * dt^2 rho[v_i]', 'rho[v_i] * dj rho[v_j]', 'rho[v_j] * dj rho[v_i]']
#inhomog_terms = ['dj rho[v_i * v_j]', 'dt di rho', 'dt dj^2 rho[v_i]', 'dt rho[v_i]', 'dt^2 di rho',
#'rho * dt^2 rho[v_i]', 'rho[v_i] * dj rho[v_j]', 'rho[v_j] * dj rho[v_i]', 'di rho']
col_numbers = [find_term(libs[1].terms, name) for name in inhomog_terms]
Xi, lambd = regress(libs[1].Q, col_numbers, libs[1].col_weights)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[1].terms[i]}. \n(Char. size: {libs[1].col_weights[i]}, term contribution: {abs(np.linalg.norm(libs[1].Q[:, i])*Xi[i])})")
print("Model residual:", lambd)

[Term 2] 0.0008745238100672925 * dt di rho. 
(Char. size: 0.024424875628643372, term contribution: 0.06668920578637524)
[Term 4] 1.0 * dt^2 di rho. 
(Char. size: 0.0024424875628643373, term contribution: 4.691589783176698)
[Term 13] -0.008076617680246194 * dt rho[v_i]. 
(Char. size: 0.01747845916705012, term contribution: 2.143535963045057)
[Term 15] 0.7824016869735495 * dt dj^2 rho[v_i]. 
(Char. size: 0.0006991383666820046, term contribution: 3.784904460110101)
[Term 22] 0.004819990279059284 * rho * dt^2 rho[v_i]. 
(Char. size: 0.0034956918776194385, term contribution: 1.5050974246969686)
[Term 24] -0.007415440308375883 * dj rho[v_i * v_j]. 
(Char. size: 0.005003039352159132, term contribution: 1.9256094472338354)
[Term 27] 0.0035371449010126463 * rho[v_i] * dj rho[v_j]. 
(Char. size: 0.0247182743160344, term contribution: 2.5780670785414346)
[Term 29] -0.003260556963014042 * rho[v_j] * dj rho[v_i]. 
(Char. size: 0.0247182743160344, term contribution: 2.4272037321809266)
Model residua

In [ ]:
noise=0.1:

noise=0.2:
1.0 * dj rho[v_i * v_j] + 0.9890065535710518 * dt rho[v_i] = 0 ; residual: 0.14585171310218614
1.0 * dj rho[v_i * v_j] + 0.9410084749613301 * dt rho[v_i] + 
    -0.2515534627028856 * rho * dj rho[v_i * v_j] + -0.23378824355993344 * rho * dt rho[v_i] = 0 ; 
    residual: 0.09879053083935584
0.1303148677799985 * di dj^2 rho + 0.16323332493962914 * dj rho[v_i * v_j] + 
    1.0 * dt di dj rho[v_j] + 0.16574805811439555 * dt rho[v_i] = 0 ; 
    residual: 0.13484770611457966 ... 
    (note: dj rho[v_j] = -dt rho, so added terms are di [c^2 dj^2 rho-dt^2 rho])
next - di rho, dj rho * rho[v_i * v_j]